# Introduction to the SIR Model
One of the canonical epidemiological models is the SIR model. Below is an interactive notebook that will allow you to explore the variables. While determining an analytical solution to the SIR model is possible, we will solve the system of differential equations using odeint([See here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html) for the documentation if you are interested)

## Table of Contents
1. [How to Use This Notebook](#how-to-use)
2. [Model Code](#model-code)
3. [Interactive Graph](#graph)

<a id='how-to-use'></a>

## How to Use This Notebook

1. Select the "Kernel" menu button
2. Select "Restart and Run All"
3. Use the sliders to control the values for $\alpha$ and $\gamma$
4. Observe how altering these values change the dynamics of the SIR plots

<a id='model-code'></a>

## Model Code

In [ ]:
## Imports
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact, interactive
import ipywidgets as widgets

In [ ]:
## Model and odeint function definition
def mm_model(y, t, beta, gamma, n):
    """ Solves system using the odeint definition and function
    
    Arguments
        y: List of variable solutions
        t: List of time steps to evaluate at
        beta: Infection rate constant
        gamma: Recovery rate constant
        n: Total number in the population
    
    Returns:
        ds: Change in S 
        di: Change in I
        dr: Change in R
        
    """
    s, i, r = y
    ds = - beta * s * i / n
    di = beta * s * i / n - gamma * i
    dr = gamma * i

    return ds, di, dr

## Starting values
beta = 0.5
gamma = 0.1
r0 = round(beta/gamma, 2)
recovery_time = round(1/gamma, 1)
y0 = [999999, 1, 0]
n = np.sum(y0)

## Define time array
t = np.linspace(0, 365, 730)

## Initial odeint solution
sol = odeint(mm_model, y0, t, args=(beta, gamma, n))

## Setup FigureWidget and initial trace
fig = go.FigureWidget()
fig.add_scatter(x=t, y=sol[:,0], name="Susceptible", line_color='red')
fig.add_scatter(x=t, y=sol[:,1], name="Infected", line_color='green')
fig.add_scatter(x=t, y=sol[:,2], name='Recovered', line_color='blue')
fig.update_layout(title='SIR Model — R<sub>0</sub> = {} | Infectious Period : {} Days'.format(r0, recovery_time),
                  template='plotly_white')
fig.update_xaxes(title='Days')
fig.update_yaxes(title='Number of Individuals')

## Define ipywidget update function
def graph_update(beta, gamma):
    """ Update graph data via ipywidgets
    
    Arguments:
        beta: Infection rate constant
        gamma: Recovery rate constant 
        
    Prints:
        r0: Calculated R0
        
    """
    sol = odeint(mm_model, y0, t, args=(beta, gamma, n))
    fig.data[0].y = sol[:,0]
    fig.data[1].y = sol[:,1]
    fig.data[2].y = sol[:,2]
    r0 = round(beta/gamma,2)
    recovery_time = round(1/gamma, 1)
    fig.update_layout(title='SIR Model — R<sub>0</sub> = {} | Infectious Period : {} Days'.format(r0, recovery_time))
    print(r0)
    
## Define Widgets and UI
beta_widget = widgets.FloatSlider(value=0.5,  min=0.01, max=2, step=0.01, description="$\\beta$")
gamma_widget = widgets.FloatSlider(value=0.1,  min=0.01, max=1, step=0.01, description="$\gamma$")
out = widgets.interactive_output(graph_update, {'beta' : beta_widget, 'gamma' : gamma_widget})
slider_ui = widgets.HBox([beta_widget, gamma_widget])

<a id='graph'></a>

## Interactive Graph

In [ ]:
display(fig, slider_ui)

### Example Questions to Explore
- How does changing $\gamma$, and hence the time a person takes to recover, alter the dynamics of the system?
- Does a disease with the same $R_0$, but different $\gamma$ have the same dynamics?